In [ ]:
import pandas as pd
import numpy as np

# Load all files
df_pop = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Population_2023.csv")
df_income = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/median_income.csv")
df_insurance = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Insurance_Coverage_Total_Population_by_state.csv")
df_broadband = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/broadband_by_state.csv")
df_facilities = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Center_Service_Delivery_and_LookAlike_Sites.csv")
df_travel = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/state_travel_time.csv")




/var/folders/vk/fzlwy9hj4ts96ppvlkx7lpmc0000gn/T/ipykernel_98360/230149653.py:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_broadband = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/broadband_by_state.csv")
/var/folders/vk/fzlwy9hj4ts96ppvlkx7lpmc0000gn/T/ipykernel_98360/230149653.py:9: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_facilities = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Center_Service_Delivery_and_LookAlike_Sites.csv")


Working with Population dataset
* Keeping the 50 states and not including purto rico
* need to strip data from state column to remove trailing spaces

In [172]:
df_pop = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Population_2023.csv")

In [202]:
# state column has trailing space so renamed it and rest is clean
# state is a string and population is int so it is good
df_pop.columns = df_pop.columns.str.lower()
df_pop.rename(columns={'state ':'state'},inplace=True)

In [253]:
df_pop['state'] = df_pop['state'].str.strip()

Working with median income dataset
* Converted the column names to lower case
* The income is object -> converted to string
* Income is divided by 1000 to make the numbers small
* changing column name of location -> state



In [217]:
df_income = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/median_income.csv")

In [219]:
df_income.rename(columns={'Location':'location','Median Annual Household Income':'median_income($1k)'},inplace=True)


In [221]:
# median income is string, need to convert it to int and also remove the $
df_income['median_income($1k)'] = (
    df_income['median_income($1k)']
    .replace('[\$,]', '', regex=True)
    .astype(float)
    / 1000
)


In [223]:
df_income = df_income[(df_income['location'] != 'United States') & (df_income['location'] != 'Puerto Rico')]

In [225]:
df_income.rename(columns={'location':'state'},inplace=True)

/var/folders/vk/fzlwy9hj4ts96ppvlkx7lpmc0000gn/T/ipykernel_98360/2017228073.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_income.rename(columns={'location':'state'},inplace=True)


In [ ]:
df_income['state'] = df_income['state'].str.strip()

Working with insurance dataset:
* removing "united states" and purto rico rows
* creating new column by using ~uninsured to get the % of insured people
* keeping only relevant columns : Location and Insured

In [ ]:
df_insurance.columns = df_insurance.columns.str.lower()
df_insurance

In [160]:
df_insurance['insured'] = (1 - df_insurance['uninsured']) * 100
df_insurance = df_insurance[['location','insured']]

In [161]:
df_insurance = df_insurance[(df_insurance['location'] != 'United States')& (df_insurance['location'] != 'Puerto Rico')]

In [163]:
df_insurance.rename(columns={'location':'state'},inplace=True)

In [255]:
df_insurance['state'] = df_insurance['state'].str.strip()

Working on Broadband dataset:
* need just two columns: state and brodband%
* cleaning the "geography_desc" column using regex to extract state
* droppping rows where state could not be extracted
* feature engineering by grouping all the states and then finding average broadband%
* mapping the State Abbervation which was extracted to the state name 

In [ ]:
df_broadband = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/broadband_by_state.csv")
df_broadband

In [ ]:
df_broadband['state_abbr'] = df_broadband['geography_desc'].str.extract(r',\s*([A-Z]{2})')
df_broadband

In [282]:
df_broadband = df_broadband.dropna(subset=['state_abbr'])

In [283]:
df_broadband = df_broadband.groupby('state_abbr')['res_st_pct'].mean().reset_index().rename(columns={'state_abbr':'state','res_st_pct':'broadband'})

In [284]:
df_broadband = df_broadband[(df_broadband['state']!='PR')&(df_broadband['state']!='GU')&(df_broadband['state']!='MP')&(df_broadband['state']!='AS')&(df_broadband['state']!='VI')]

In [ ]:
df_broadband.reset_index(drop=True)

In [286]:
df_broadband['broadband'] = (df_broadband['broadband']*100).round(2)

In [287]:
us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut','DC':'District of Columbia' ,'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

df_broadband['state'] = df_broadband['state'].map(us_state_abbrev)

In [288]:
df_broadband['state'] = df_broadband['state'].str.strip()

In [ ]:
df_broadband

Working with Facilities dataset:
* filter and keep only column = Site State Abbreviation
* aggregate that column and count the number of occurances and put it in new column

In [294]:
df_facilities = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Center_Service_Delivery_and_LookAlike_Sites.csv")


/var/folders/vk/fzlwy9hj4ts96ppvlkx7lpmc0000gn/T/ipykernel_98360/894847931.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_facilities = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Center_Service_Delivery_and_LookAlike_Sites.csv")


In [295]:
df_facilities = df_facilities[['Site State Abbreviation']]

In [296]:
#dropping nan values
df_facilities = df_facilities.dropna(subset=['Site State Abbreviation'])
df_facilities.rename(columns={'Site State Abbreviation':'state'},inplace=True)

In [297]:

df_facilities['state'] = df_facilities['state'].map(us_state_abbrev)

In [299]:
df_facilities = df_facilities.groupby('state').size().reset_index()

In [300]:
df_facilities.rename(columns={0:'facility_count'},inplace=True)

In [301]:
df_facilities['state'] = df_facilities['state'].str.strip()

Working with state travel time dataset:
* converting column names to lower case
* associating the state abbrevations to full name 

In [303]:
df_travel = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/state_travel_time.csv")


In [304]:
df_travel.columns = df_travel.columns.str.lower()

In [305]:
df_travel['state'] = df_travel['state'].map(us_state_abbrev)

In [306]:
df_travel['state'] = df_travel['state'].str.strip()

Merging the clean dataset

In [188]:
df_final = df_pop.copy()

In [199]:
df_pop.columns

Index(['state ', 'population_2023'], dtype='object')

In [308]:
# Start with population
df_final = df_pop.copy()

# Merge income
df_final = df_final.merge(df_income, on='state', how='left')

# Merge insurance
df_final = df_final.merge(df_insurance, on='state', how='left')

# Merge broadband (state in lowercase)
df_final = df_final.merge(df_broadband, on='state', how='left')

# Merge health facility count
df_final = df_final.merge(df_facilities, on='state', how='left')

# Optional: Merge travel time
# df_final = df_final.merge(df_travel_clean, on='State', how='left')

# Final check



In [309]:
df_final.to_csv('/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/clean/healthcare_accessibility_index.csv', index=False)

In [ ]:
df_pop['state']

In [263]:
df_broadband[df_broadband['state']=='District of Columbia']

,state,broadband


In [310]:
df_final


,state,population_2023,median_income($1k),insured,broadband,facility_count
0,California,39576757,95.521,93.6,61.99,2867
1,Texas,29994331,75.780,83.7,66.43,821
2,Florida,22377419,73.311,89.3,67.23,808
3,New York,19530351,82.095,95.2,75.42,882
4,Pennsylvania,12804123,73.824,94.6,73.86,479
5,Illinois,12585336,80.306,93.9,69.35,534
6,Ohio,11780017,67.769,93.9,66.83,599
7,Georgia,10942800,74.632,88.6,66.86,444
8,North Carolina,10701022,70.804,90.8,68.50,744
9,Michigan,10050811,69.183,95.6,61.70,478
